In [1]:
library(ggplot2)
library(ggExtra)
library(brms)
library(reshape2)
library(coda)
library(tidybayes)
library(ggstance)
library(viridis)
library(latex2exp)
library(ggthemes)
library(data.table)

Loading required package: Rcpp
Loading 'brms' package (version 2.4.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').
Run theme_set(theme_default()) to use the default bayesplot theme.
NOTE: As of tidybayes version 1.0, several functions, arguments, and output column names
      have undergone significant name changes in order to adopt a unified naming scheme.
      See help('tidybayes-deprecated') for more information.


Attaching package: 'ggstance'

The following objects are masked from 'package:ggplot2':

    geom_errorbarh, GeomErrorbarh

Loading required package: viridisLite

Attaching package: 'data.table'

The following objects are masked from 'package:reshape2':

    dcast, melt



In [3]:
get_data <- function(){
    filename <- "C:/Users/Arkady/Google Drive/data/beyond_the_reach/k-values.txt"
    data <- read.table(filename, header = TRUE, sep = "\t")
    data[, 'subj_id'] <- factor(data[, 'subj_id'])  
    data[, 'task'] <- factor(data[, 'task'])  
    data[, 'order'] <- factor(data[, 'order'])  
    return(data)
}

data <- get_data()
nrow(data)

[1] 102

In [11]:
get_bf_k <- function(data){
    # sd(IV) in this case is 0.5, as IV is task (equal number of 0's and 1's)
    priors_task <- c(set_prior(sprintf('normal(%f, %f)', mean(data$k), sd(data$k)), class = 'Intercept'),
                    set_prior(sprintf('cauchy(0.0, %f)', 0.707*sd(data$k)/0.5), class = 'b'))

    m_null <- brm(k ~ (1 | subj_id), data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task[1,],
                  refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))
    m_task <- brm(k ~ (1 | subj_id) + task, data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task, 
                  refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))    
    m_order <- brm(k ~ (1 | subj_id) + order, data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task, 
                  refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))
#     m_inter <- brm(k ~ (1 | subj_id) + task*order, data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task, 
#                   refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))
    
    bf_task <- bayes_factor(x1=m_task, x2=m_null)$bf
    bf_order <- bayes_factor(x1=m_order, x2=m_null)$bf
#     bf_inter <- bayes_factor(x1=m_inter, x2=m_null)$bf
    names(bf_task) <- 'bf_task'
    names(bf_order) <- 'bf_order'
#     names(bf_inter) <- 'bf_inter'
        
    result = list(bf=t(c(bf_task, bf_order)), m_null=m_null, m_task=m_task, m_order=m_order)
    
    return(result)
}

In [12]:
result <- get_bf_k(data)

Compiling the C++ model
Start sampling



Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.401 seconds (Warm-up)
               0.167 seconds (Sampling)
               0.568 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.405 seconds (Warm-up)
               0.16 seconds (Sampling)
               0.565 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.365 seconds (Warm-up)
               0.161 seconds (Sampling)
               0.526 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.465 seconds (

Compiling the C++ model
Start sampling



Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.519 seconds (Warm-up)
               0.241 seconds (Sampling)
               0.76 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.532 seconds (Warm-up)
               0.23 seconds (Sampling)
               0.762 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.583 seconds (Warm-up)
               0.227 seconds (Sampling)
               0.81 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.553 seconds (Wa

Compiling the C++ model
Start sampling



Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.754 seconds (Warm-up)
               0.258 seconds (Sampling)
               1.012 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.551 seconds (Warm-up)
               0.252 seconds (Sampling)
               0.803 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.66 seconds (Warm-up)
               0.304 seconds (Sampling)
               0.964 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.57 seconds (W

In [14]:
1/result$bf

bf_task,bf_order
14.66627,5.074242


In [22]:
hpd_task<-HPDinterval(as.mcmc(result$m_task, combine_chains = TRUE))
hpd_order<-HPDinterval(as.mcmc(result$m_order, combine_chains = TRUE))

In [23]:
hpd_order['b_orderwm',]
hpd_task['b_taskwalking',]

lower       upper 
-0.13304299  0.07514511

lower       upper 
-0.04776541  0.02684763